# Osnovna analiza podatkov

V tem predavanju bomo obravnavali osnovne funkcije za analizo podatkov, ki smo jih uvozili in spravili v format `tidy_data`. Spoznali jih bomo na primerih njihove uporabe za analizo podatkovne tabele `pt` z umetnimi podatki. Podatkovno tabelo bomo sestavili iz treh stolpcev (spremenljivk), `d`, `c` in `t`, ki bodo datumskega, kategoričnega in numeričnega tipa, z naključnimi vrednostmi.

Nato bomo iste funkcije uporabili za analizo podatkov o dosežkih študentov FMF, ki smo jih pripravili na prejšnjih predavanjih.

Vse funkcije, ki jih bomo obravnavali na tem predavanju delujejo na zaporedjih (_series_) in podatkovnih tabelah (_dataframe_). Mi bomo vedno uporabljali različice za zaporedja. Uporaba na tabelah je identična ali zelo podobna. Vsaka funkcija ima dvojno dokumentacijo, po eno stran za vsako različico. Tako na primer, lahko preberemo dokumentacijo za različico funkcije [`mean` za zaporedja](https://pandas.pydata.org/docs/reference/api/pandas.Series.mean.html) ali pa [za podatkovne tabele](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mean.html).

In [90]:
import pandas as pd
from random import seed, sample, randint

n = 20
seed(42)

def nakljucni_datumi(datum_od, datum_do, n):
    ndni = (datum_do - datum_od).days + 1
    return datum_od + pd.to_timedelta([randint(0, ndni) for _ in range(n)], unit = "D")
ds = nakljucni_datumi(pd.to_datetime("2023-10-01"), pd.to_datetime("2023-11-30"), n)

def nakljucne_kategorije(kategorije, n):
    return sample(kategorije, n, counts = [n for _ in kategorije])
ks = pd.Series(nakljucne_kategorije(["a", "b", pd.NA], n), dtype = "category")

xs = [x if abs(x) < 100 else None for x in sample(range(-125, 125), n)]

pt = pd.DataFrame({"d": ds, "k": ks, "x": xs})
print(pt)

            d    k     x
0  2023-11-10    a -54.0
1  2023-10-08    a -86.0
2  2023-10-02    b -70.0
3  2023-11-17    b   NaN
4  2023-10-18    a  70.0
5  2023-10-16    b -39.0
6  2023-10-15    a -99.0
7  2023-10-09  NaN   NaN
8  2023-11-17  NaN -28.0
9  2023-10-07  NaN   NaN
10 2023-11-13    b -34.0
11 2023-11-17    b  91.0
12 2023-11-27  NaN -37.0
13 2023-11-04    b  29.0
14 2023-10-06    b -58.0
15 2023-11-07    a  81.0
16 2023-10-28    a   NaN
17 2023-10-03    a  61.0
18 2023-10-02    b  -8.0
19 2023-10-06    b  12.0


## Osnovne statistike

Spoznajmo najprej funkcije, ki so definirane na zaporedjih in nam omogočajo izračun osnovnih agregatnih statistik vrednosti v zaporedju. Prvi argument vseh teh funkcij je zaporedje, vse jih lahko uporabljamo oziroma kličemo kot metode objekta tipa zaporedje.

| Funkcija | Opis delovanja |
|:---|:---|
| `count` |	Število _znanih_ vrednosti |
| `nunique` | Število _različnih znanih_ vrednosti |
| `sum` | Vsota _znanih_ vrednosti |
| `mean` | Povprečna vrednost |
| `mad` | Povprečno absolutno odstopanje |
| `median` | Mediana, srednja vrednost |
| `min` | Minimalna vrednost |
| `max` | Maksimalna vrednost |
| `quantile` | Kvantil za podan procent, npr. pri 50% dobimo mediano |
| `value_counts` | Število pojavitev različnih vrednosti |
| `mode` | Modus, najbolj pogosta vrednost |
| `prod` | Produkt vrednosti |
| `std` | Standardni odklon, nepristranski |
| `var` | Standardna variance, nepristranska |
| `skew` | Asimetrija, nepristranska, tretji moment |
| `kurt` | Sploščenost, nepristranska, četrti moment |
| `cumsum` | Kumulativne vsote |
| `cumprod` | Kumulativni produkti |
| `cummax` | Kumulativne maksimalne vrednosti |
| `cummin` | Kumulativne minimalne vrednosti |

Poglejmo si primere uporabe zgornjih funkcij:

In [67]:
print(f"Stolpec x ima {pt.x.count()} znanih vrednosti.")
print(f"Njihova vsota je {pt.x.sum()}, povprečje {pt.x.mean():.4}, mediana pa {pt.x.median()}.")

Stolpec x ima 16 znanih vrednosti.
Njihova vsota je -55.0, povprečje -3.438, mediana pa -1.5.


Podobne funkcije lahko uporabimo tudi za datumsko spremenljivko:

In [91]:
print(f"Datumi v d so od {pt.d.min()} do {pt.d.max()}.")
print(f"Povprečna vrednost d je {pt.d.mean()}, srednja pa {pt.d.median()}.")

Datumi v d so od 2023-10-02 00:00:00 do 2023-11-27 00:00:00.
Povprečna vrednost d je 2023-10-24 00:00:00, srednja pa 2023-10-17 00:00:00.


Poglejmo zakaj srednja vrednost pade ravno ob poldne. Pri računanju mediane, namreč, najprej uredimo vrednosti spremenljivke `d` v naraščajočem ali padajočem vrstnem redu:

In [92]:
print(pt.d.sort_values())


2    2023-10-02
18   2023-10-02
17   2023-10-03
19   2023-10-06
14   2023-10-06
9    2023-10-07
1    2023-10-08
7    2023-10-09
6    2023-10-15
5    2023-10-16
4    2023-10-18
16   2023-10-28
13   2023-11-04
15   2023-11-07
0    2023-11-10
10   2023-11-13
8    2023-11-17
11   2023-11-17
3    2023-11-17
12   2023-11-27
Name: d, dtype: datetime64[ns]


V tem urejenem seznamu nas zanima katera vrednost je na sredini tega zaporedja. V primeru zaporedja z liho dolžino, je zgolj en element točno v sredini, zato je tudi vrednost mediane enaka vrednosti tega elementa. V tem primeru, ko ima zaporedje sodo število elementov, sta dva elementa (s pozicijskima indeksoma `9` in `10`) na sredini tega seznama. Mediano oziroma srednjo vrednost v tem primeru izračunamo takole:

In [93]:
print(pt.d.sort_values().iloc[[9,10]].mean())

2023-10-17 00:00:00


Srednja vrednost je kvantil zaporedja pri 50%: je vrednost elementa (ali povprečna vrednost elementov), ki je na taki poziciji (pozicijah) v urejenem zaporedju, ki predstavlja polovico (50%) dolžine zaporedja. Funkcija `quantile` izračuna kvantil za poljubno podano vrednost v odstotkih:

In [94]:
print("\n".join([f"Q(pt.x, {p}) = {pt.x.quantile(p)}" for p in [n / 4 for n in range(5)]]))

Q(pt.x, 0.0) = -99.0
Q(pt.x, 0.25) = -55.0
Q(pt.x, 0.5) = -31.0
Q(pt.x, 0.75) = 37.0
Q(pt.x, 1.0) = 91.0


Za vajo ugotovi, kako iz zaporedja ali seznama urejenih vrednosti zaporedja `pt.x` izračunamo zgornje kvantile.

Na koncu si poglejmo še funkcijo `mode`, ki izračuna modus, t.j., najbolj zastopano vrednost v zaporedju tipa kategorija:

In [111]:
print(pt.k.mode())

k
b    9
a    7
Name: count, dtype: int64
0    b
Name: k, dtype: category
Categories (2, object): ['a', 'b']


Za kategorično zaporedje `pt.k` poskušaj izračunati srednjo ali povprečno vrednost. Pojasni rezultat poskusa. Preveri spremembe rezultata, če bi imeli zaporedje tipa urejena kategorija.

Za vajo preizkusi delovanje vseh funkcij iz tabele na začetku tega razdelka.

## Preverjanje in spreminjanje vrednosti

Preden nadaljujemo, poglejmo na hitro še nekaj funkcij, ki nam omogočajo preverjanje in spreminjanje (nastavljanje) vrednosti elementov zaporedja ali podatkovne tabele.

Pogosto uporabljena funkcija je `isna`, ki za podano zaporedje vrne zaporedje Boolovih vrednosti. Vrednost elementa rezultata, ki je enaka `True`, nakazuje, da ima isto ležeči element podanega zaporedja _neznano_ vrednost, vrednost element `False`, da ima _znano_ vrednost. Poglejmo:

In [102]:
print(pt.x.isna())

0     False
1     False
2     False
3      True
4     False
5     False
6     False
7      True
8     False
9      True
10    False
11    False
12    False
13    False
14    False
15    False
16     True
17    False
18    False
19    False
Name: x, dtype: bool


Če seštejemo vrednosti dobljenega zaporedja, dobimo število neznanih vrednosti za podano zaporedje. Ker je to lahko uporabno, bi lahko napisali funkcijo `n_na`, ki za podano zaporedje vrne število neznanih vrednosti v zaporedju:

In [109]:
def n_na(s):
    return s.isna().sum()

print(f"{n_na(pt.d)=}, {n_na(pt.k)=}, {n_na(pt.x)=}")

n_na(pt.d)=0, n_na(pt.k)=4, n_na(pt.x)=4


Za vajo definiraj funkcijo `n_na` brez uporabe funkcije `isna`, zgolj s pomočjo funkcij iz prejšnjega razdelka (namig: potrebuješ funkcijo `count`).

Pogosto smo v situaciji, ko neznane vrednosti zaporedja moramo nadomestiti z nekimi znanimi vrednostmi. To nam omogoča funkcija [`fillna`](https://pandas.pydata.org/docs/reference/api/pandas.Series.fillna.html), ki v najbolj enostavni različici nadomesti neznane vrednosti z vnaprej podano, fiksno vrednostjo, npr. `0`:

In [112]:
print(pt.x.fillna(0))

0    -54.0
1    -86.0
2    -70.0
3      0.0
4     70.0
5    -39.0
6    -99.0
7      0.0
8    -28.0
9      0.0
10   -34.0
11    91.0
12   -37.0
13    29.0
14   -58.0
15    81.0
16     0.0
17    61.0
18    -8.0
19    12.0
Name: x, dtype: float64


Poglejmo zdaj kako se spremeni povprečna vrednost v podanem zaporedju, če neznane vrednosti zaporedja nadomestimo z `0`:

In [113]:
print(pt.x.mean(), pt.x.fillna(0).mean())

-10.5625 -8.45


Za vajo pojasni nastalo spremembo povprečne vrednosti. Kako `mean` izračuna povprečno vrednost elementov zaporedja, ki vsebuje neznane vrednosti?

Bolj splošna funkcija za nadomeščanje (zamenjavo) vrednosti je [`replace`](https://pandas.pydata.org/docs/reference/api/pandas.Series.replace.html), ki (spet v osnovni različici) podano vrednost nadomesti z drugo podano vrednostjo:

In [114]:
print(pt.k.replace("a", "A"))

0       A
1       A
2       b
3       b
4       A
5       b
6       A
7     NaN
8     NaN
9     NaN
10      b
11      b
12    NaN
13      b
14      b
15      A
16      A
17      A
18      b
19      b
Name: k, dtype: category
Categories (2, object): ['A', 'b']


Funkciji lahko podamo kot argumenta seznama enakih dolžin, tako kot tukaj:

In [115]:
print(pt.k.replace(["a", "b"], ["A", "B"]))

0       A
1       A
2       B
3       B
4       A
5       B
6       A
7     NaN
8     NaN
9     NaN
10      B
11      B
12    NaN
13      B
14      B
15      A
16      A
17      A
18      B
19      B
Name: k, dtype: category
Categories (2, object): ['A', 'B']


Funkcija `replace` je še posebej uporabna za zaporedja nizov znakov, ker sta v tem primeru lahko oba argumenta regularna izraza (ali enako dolga seznama regularnih izrazov).

Še več fleksibilnosti pri zamenjavi vrednosti elementov zaporedja ponuja funkcija [`where`](https://pandas.pydata.org/docs/reference/api/pandas.Series.where.html), kjer za izbiro elementov lahko uporabljamo poljuben logični izraz. Če za opazovani element zaporedja ima podani izraz vrednost `True`, ostane vrednost elementa nespremenjena. Sicer pa se zamenja s podano vrednostjo. Poglejmo enostaven primer:

In [124]:
print(pt.x.where(abs(pt.x) < 50, "!"))

0        !
1        !
2        !
3        !
4        !
5    -39.0
6        !
7        !
8    -28.0
9        !
10   -34.0
11       !
12   -37.0
13    29.0
14       !
15       !
16       !
17       !
18    -8.0
19    12.0
Name: x, dtype: object


Zgornji rezultat kaže na obnašanje logičnega izraza v funkciji `where`: če je vrednost elementa zaporedja `pt.x` neznana, potem izraz `abs(pt.x) < 50` nima vrednosti `True`. Zato so v rezultatu vse neznane vrednosti zamenjane s klicajem.

## Skupine elementov oziroma vrstic

Izračun statistik je že sam po sebi uporaben. A pogostokrat nas ne zanimajo vrednosti agregatnih statistik za vse elemente zaporedja ali vse vrstice podatkovne tabele, temveč bi želeli agregatne statistike izračunati za skupine vrstic. Na primer, za dosežke študentov na kolokvijih nas zanima povprečna vrednost dosežka za vsakega študenta posebej. Podobno v tabeli z umetnimi podatki iz prejšnji razdelkov, bi nas lahka zanimalo kakšna je povprečna vrednost spremenljivke `x` za različne kategorije `k` ali za različne mesece iz datumov v `d`.

Take izračune na podani podatkovni tabeli lahko opravimo z uporabo funkcije [`groupby`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html). Rezultat funkcije je objekt razreda `GroupBy`, ki razdeli vrstice v tabeli na osnovi vrednosti podane spremenljivke (ali kombinacije spremenljivk podane kot seznam): v eni skupini so vse vrstice z enako vrednostjo podane spremenljivke, skupin pa je toliko, kot je različnih vrednosti podane spremenljivke.  Če na tem objektu izračunamo agregatno statistiko, bo njen izračun izveden za vsako skupino posebej.

Poglejmo enostaven primer:

In [128]:
pt_k = pt.groupby("k")

print(type(pt_k))
print(pt_k.groups)

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>
{'a': [0, 1, 4, 6, 15, 16, 17], 'b': [2, 3, 5, 10, 11, 13, 14, 18, 19]}


/var/folders/g7/xgt9ckmj6g1404knwg8j5rkc0000gn/T/ipykernel_99367/2589392432.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pt_k = pt.groupby("k")


Rezultat `pt_k` funkcije `groupby` je torej objekt razreda `DataFrameGroupBy` z atributom `groups`, katerega vrednost je slovar, ki določa razvrstitev vrstic tabele `pt` po vrednostih spremenljivke `k`. Ker ima slednja dve možni vrednosti, ima slovar dva ključa, ki ustrezata tem vrednostim. Vrednost pri izbranem ključu je seznam indeksov tistih vrstic podane tabele v katerih je vrednost spremenljivke `k` enaka vrednosti izbranege ključa. **Pozor**: neznane vrednosti ne tvorijo nobene skupine.

Prava moč funkcije pride do izraza pri izračunu agregatnih statistik:

In [129]:
print(pt_k.x.mean())

k
a   -4.500
b   -9.625
Name: x, dtype: float64


Rezultat je torej zaporedje povprečnih vrednosti spremenljivke `x` za različne vrednosti spremenljivke `k`.

Funkcija [`agg`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.agg.html) nam omogoča več fleksibilnosti pri izračunu agregatnih statistik. Lahko izračunamo več agregatnih statistik za različne spremenljivke podane podatkovne tabele:

In [134]:
print(pt_k.agg({
    "d": ["min", "max"],
    "x": ["mean", "std"]
}))

           d                 x           
         min        max   mean        std
k                                        
a 2023-10-03 2023-11-10 -4.500  83.873118
b 2023-10-02 2023-11-17 -9.625  52.820282


V zgornjem primeru smo kot argument funkcije `agg` podali slovar, kjer so ključi spremenljivke podatkovne tabele, vrednosti pa seznami imen agregatnih funkcij iz prejšnjega razdelka. Pri tem nimamo dobre kontrole nad imeni stolpcev v rezultati. Če si to želimo, bi lahko podali malo drugačen argument funkciji `agg`:

In [140]:
print(pt_k.agg(
    datum_od = ("d", "min"),
    datum_do = ("d", "max"),
    povprecje_x = ("x", "mean"),
    odklon_x = ("x", "std")
))

    datum_od   datum_do  povprecje_x   odklon_x
k                                              
a 2023-10-03 2023-11-10       -4.500  83.873118
b 2023-10-02 2023-11-17       -9.625  52.820282


Skupine vrstic v tabeli lahko določamo tudi na osnovi vrednosti numerične spremenljivke. V takih primerih argument običajno podamo v obliki logičnega izraza in dobimo dve skupini. V eni skupini so vrstice, kjer je vrednost podanega logičnega izraza `True`, v drugi pa vse ostale vrstice (bodimo pozorni na neznane vrednosti, tako kot pri funkciji `where`):

In [144]:
print(pt.groupby(abs(pt.x) < 50).groups)

{False: [0, 1, 2, 3, 4, 6, 7, 9, 11, 14, 15, 16, 17], True: [5, 8, 10, 12, 13, 18, 19]}


Podobno lahko skupine določamo tudi na osnovi datuma. Na tem mestu nam pridejo prav različne funkcije za izračune derivatov datuma, kot so mesec, teden ali pa dan v tednu (za osvežitev spomina, prelistaj zapiske s predavanj iz začetka semestra, kjer smo obravnavali različne tipe zaporedij):

In [155]:
print(pt.groupby(pt.d.dt.month_name()).mean("x"))

                  x
d                  
November   6.857143
October  -24.111111


Izračunali smo torej povprečne vrednosti spremenljivke `x` za vse mesece datumov v spremenljivki `d`. Lahko bi podobno izračunali povprečne vrednosti `x` za različne dneve v tednu:

In [159]:
print(pt.groupby(pt.d.dt.day_name()).mean("x"))

              x
d              
Friday     -7.4
Monday    -37.6
Saturday   29.0
Sunday    -92.5
Tuesday    71.0
Wednesday  70.0


Funkcija `groupby` nam omogoča tudi določanje skupin na osnovi različnih kombinacij vrednosti spremenljivk. Ugotovi na primer, kaj so skupine v naslednjem primeru:

In [167]:
print(pt.groupby(["k", pt.d.dt.day_name()]).groups)

{('a', 'Friday'): [0], ('a', 'Saturday'): [16], ('a', 'Sunday'): [1, 6], ('a', 'Tuesday'): [15, 17], ('a', 'Wednesday'): [4], ('b', 'Friday'): [3, 11, 14, 19], ('b', 'Monday'): [2, 5, 10, 18], ('b', 'Saturday'): [13], (nan, 'Friday'): [8], (nan, 'Monday'): [7, 12], (nan, 'Saturday'): [9]}


/var/folders/g7/xgt9ckmj6g1404knwg8j5rkc0000gn/T/ipykernel_99367/1087440675.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(pt.groupby(["k", pt.d.dt.day_name()]).groups)


Zanimivo je, da tokrat rezultat vključuje tudi neznane vrednosti spremenljivke `k`. Za vajo ugotovi kako se znebiti opozorila (FutureWarning), ki se izpiše med izvajanjem te kode.

## Analiza podatkov o rezultatih kolokvijev

V enem od prejšnjih predavanjih smo urejali podatke o rezultatih kolokvijev. Spomnimo se, da smo jih uredili v dve podatkovni tabeli:

  * `studenti` s podatki o posameznih študentih (vrstice) z eno dimenzijsko spremenljivko `ime` in tri vrednostne spremenljivke `telefon`, `spol` in `starost`.

  * `studenti_ocene` s podatki o dosežkih študentov na kolokvijih, ki bo vsebovala tri dimenzijske spremenljivke `student`, `semester` in `kolokvij` ter eno vrednostno spremenljivko `rezultat`. Za izbranega študenta (izbrano študentko), bo tabela podala njegov (njen) dosežen rezultat na izbranem kolokviju v izbranem semestru.

Tabeli preberemo iz datotek [`studenti.csv`](https://kt.ijs.si/~ljupco/lectures/papvp-2324/studenti.csv) in [`studenti_ocene.csv`](https://kt.ijs.si/~ljupco/lectures/papvp-2324/studenti_ocene.csv):

In [194]:
studenti = pd.read_csv("https://kt.ijs.si/~ljupco/lectures/papvp-2324/studenti.csv")
studenti_ocene = pd.read_csv("https://kt.ijs.si/~ljupco/lectures/papvp-2324/studenti_ocene.csv")

print(f"studenti =\n{studenti}\n\nstudenti_ocene =\n{studenti_ocene}")

studenti =
      ime  telefon spol  starost
0     Ana      431    ž       19
1  Branko      720    m       20
2  Cvetka      761    ž       21
3   David      141    m       19
4     Eva      210    ž       20
5   Franc      592    m       21

studenti_ocene =
       ime  semester  kolokvij  rezultat
0      Ana         1         1      80.0
1      Ana         1         2      82.0
2      Ana         2         1      97.0
3      Ana         2         2      95.0
4   Branko         1         1      78.0
5   Branko         1         2       NaN
6   Branko         2         1      74.0
7   Branko         2         2       NaN
8   Cvetka         1         1       NaN
9   Cvetka         1         2      63.0
10  Cvetka         2         1       NaN
11  Cvetka         2         2      87.0
12   David         1         1      75.0
13   David         1         2      92.0
14   David         2         1      68.0
15   David         2         2      81.0
16     Eva         1         1      63.0
17

### Študenti, ki so opravljali posamezen kolokvij

Izračunajmo najprej koliko študentov je pisalo posamezen kolokvij. Nato pa za vsak kolokvij izpišimo študente, ki so h kolokviju pristopili. Posamezen kolokvij dobimo tako, da razdelimo vrstice podatkovne tabele `studenti_ocene` v skupine na osnovi kombinacije vrednosti spremenljivk `semester` in `kolokvij`. Ker imata obe spremenljivki isto domeno $\{1, 2\}$ z dvema elementoma, bomo dobili štiri skupine vrstic, za vsako kombinacijo vrednosti po eno:

  * Prva skupina bo ustrezala kombinaciji `semester = 1, kolokvij = 1`, torej prvemu kolokviju v prvem semestru;

  * Druga skupina bo ustrezala kombinaciji `semester = 1, kolokvij = 2`;

  * Tretja kombinaciji `semester = 2, kolokvij = 1`;

  * Četrta pa kombinaciji `semester = 2, kolokvij = 2`.

Število študentov, ki so kolokvij opravljali dobimo tako, da v skupini vrstic za ta kolokvij preštejemo koliko je takih, da ima spremenljivka `rezultat` znano vrednost:

In [195]:
print(
    studenti_ocene.groupby(["semester", "kolokvij"]).agg(
        st_studentov = ("rezultat", "count")
    )
)

                   st_studentov
semester kolokvij              
1        1                    5
         2                    5
2        1                    5
         2                    5


Na vsakem kolokviju je torej pristopilo pet študentk in študentov. Ali lahko dobimo sezname študentov in študentk, ki so pristopili h kolokviju:

In [196]:
skupine = studenti_ocene.groupby(["semester", "kolokvij"]).groups

print({k: studenti_ocene.loc[skupine[k], "ime"].to_list() for k in skupine})

{(1, 1): ['Ana', 'Branko', 'Cvetka', 'David', 'Eva', 'Franc'], (1, 2): ['Ana', 'Branko', 'Cvetka', 'David', 'Eva', 'Franc'], (2, 1): ['Ana', 'Branko', 'Cvetka', 'David', 'Eva', 'Franc'], (2, 2): ['Ana', 'Branko', 'Cvetka', 'David', 'Eva', 'Franc']}


### Povprečni rezultati študentov na kolokvijih

Izračunajmo zdaj povprečno število točk, ki jih je posamezen študent dosegel na kolokvijih. Ker gre za enostavno združevanje podatkov s povprečjem po skupinah, ki jih določa ime študenta, je rešitev, na prvi pogled, zelo enostavna:

In [197]:
print(studenti_ocene.groupby("ime").mean("rezultat"))

        semester  kolokvij  rezultat
ime                                 
Ana          1.5       1.5      88.5
Branko       1.5       1.5      76.0
Cvetka       1.5       1.5      75.0
David        1.5       1.5      79.0
Eva          1.5       1.5      72.5
Franc        1.5       1.5      82.5


Rezultat ni tak, kot bi ga pričakovali. Najprej, ima dva odvečna stolpca `semester` in `kolokvij` (za vajo premisli zakaj) in sta povprečna rezultata za Branka in Cvetko, ki sta pisala vsak po dva kolokvija manj kot ostali, napačen oziroma neprimerljiv s povprečnimi rezultati študentov, ki so pisali vse štiri kolokvije.

Prvo težave lahko rešimo s preprostim indeksiranjem:

In [198]:
print(studenti_ocene.groupby("ime").mean("rezultat").loc[:, ["rezultat"]])

        rezultat
ime             
Ana         88.5
Branko      76.0
Cvetka      75.0
David       79.0
Eva         72.5
Franc       82.5


Drugo težavo odpravimo tako, da manjkajoče vrednosti pri Branku in Cvetki nadomestimo z vrednostjo nič:

In [199]:
print(studenti_ocene.fillna(0).groupby("ime").mean("rezultat").loc[:, ["rezultat"]])

        rezultat
ime             
Ana         88.5
Branko      38.0
Cvetka      37.5
David       79.0
Eva         72.5
Franc       82.5


V zgornji kodi smo bili malo neprevidni in nadomestili vse manjkajoče vrednosti v celotni podatkovni tabeli `studenti_ocene` z nič. Lahko bi bili bolj previdni takole:

In [200]:
so_1 = studenti_ocene.copy()
so_1.rezultat = so_1.rezultat.fillna(0)
print(so_1.groupby("ime").mean("rezultat").loc[:, ["rezultat"]])

        rezultat
ime             
Ana         88.5
Branko      38.0
Cvetka      37.5
David       79.0
Eva         72.5
Franc       82.5


Do zdaj smo gotovo že opazili, da v dobljenih tabelah postanejo vrednosti spremenljivke za določanje skupin (poimenovani) indeksi vrstic. Za vajo pretvori zadnjo tabelo v podatkovno tabelo z dvema stolpcema, `ime` in `rezultat`, kjer so indeksi vrstic njihove zaporedne številke.

### Tabela z rezultati kolokvijev ob koncu študijskega leta

Zdaj pa si predstavljajmo, da rabi profesorica podatkovno tabelo, kjer bi vsaka vrstica ustrezala eni študentki (študentu) in bi vsebovala njeno (njegovo) ime, telefon ter dosežke na štirih kolokvijih v stolpcih z imeni `kolokvij_1` do `kolokvij_4`.

Najprej bi se lotili izračuna zaporedne številke kolokvija, ki je del imena štirih novih stolpcev omenjenih zgoraj. Zaporedno številko kolokvija izračunamo s kombinacijo vrednosti spremenljivk `semester` in `kolokvij`. Hkrati z računanjem zaporedne številke, zamenjamo neznane vrednosti rezultatov z ničlami ter iz tabele z rezultati izberemo le tri stolpce, ki nas zanimajo za nadaljnjo pripravo naše končne tabele z rezultati:

In [214]:
so_1 = studenti_ocene.copy()
so_1["zap_st_kolokvija"] = (so_1.semester - 1) * 2 + so_1.kolokvij
so_1.rezultat = so_1.rezultat.fillna(0)
so_1 = so_1.loc[:, ["ime", "zap_st_kolokvija", "rezultat"]]

print(so_1)

       ime  zap_st_kolokvija  rezultat
0      Ana                 1      80.0
1      Ana                 2      82.0
2      Ana                 3      97.0
3      Ana                 4      95.0
4   Branko                 1      78.0
5   Branko                 2       0.0
6   Branko                 3      74.0
7   Branko                 4       0.0
8   Cvetka                 1       0.0
9   Cvetka                 2      63.0
10  Cvetka                 3       0.0
11  Cvetka                 4      87.0
12   David                 1      75.0
13   David                 2      92.0
14   David                 3      68.0
15   David                 4      81.0
16     Eva                 1      63.0
17     Eva                 2      82.0
18     Eva                 3      82.0
19     Eva                 4      63.0
20   Franc                 1      95.0
21   Franc                 2      99.0
22   Franc                 3      82.0
23   Franc                 4      54.0


Zdaj pa moramo vrednosti stolpca `rezultat` razdeliti v štiri nove stolpce na osnovi zaporedne številke kolokvija. To nam omogoča funkcija [`pivot`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot.html) z naslednjimi argumenti:

  * `frame` je ime podatkovne tabele, ki jo hočemo spremeniti (v spodnjem primeru funkcijo kličemo kot metodo razreda podatkovna tabela);

  * `index` je ime stolpca, nabor ali seznam imen stolpcev, ki identificirajo vrstice, in katerih vrednosti se ne bodo spreminjale;

  * `columns` je ime stolpca, nabor ali seznam imen stolpcev, ki vsebujejo vrednosti za tvorjenje imen novih stolpcev;

  * `values` je ime stolpca, katerega vrednosti bodo postali (se bodo porazdelili med) vrednosti novih stolpcev.

 Argument `columns` nastavimo na `zap_st_kolokvija`, saj so imena novih stolpcev zasnovana na zaporednih številkah kolokvijev, ki jim dodamo predpono `kolokvij_` (to opravimo v vrstici, ki sledi klicu metode `pivot`). Argument `values` nastavimo na `rezultat`, t.j. ime stolpca, ki vsebuje vrednosti razdeljene med nove stolpce:

In [238]:
so_objava = so_1.pivot(
    index = "ime",
    columns = "zap_st_kolokvija",
    values = "rezultat"
)
so_objava = so_objava.add_prefix("kolokvij_")

# izračun povprečij po vrsticah
so_objava["povprecje"] = so_objava.mean(axis = 1)

# poskrbimo za dodajanje telefonskih številk
so_objava = so_objava.merge(studenti, on = "ime", how = "left")
so_objava = so_objava.loc[:, ["ime", "telefon", "kolokvij_1", "kolokvij_2", "kolokvij_3", "kolokvij_4"]]

print(so_objava)

      ime  telefon  kolokvij_1  kolokvij_2  kolokvij_3  kolokvij_4
0     Ana      431        80.0        82.0        97.0        95.0
1  Branko      720        78.0         0.0        74.0         0.0
2  Cvetka      761         0.0        63.0         0.0        87.0
3   David      141        75.0        92.0        68.0        81.0
4     Eva      210        63.0        82.0        82.0        63.0
5   Franc      592        95.0        99.0        82.0        54.0


S klicem funkcije (metode) `mean` smo izračunali povprečja posameznih vrstic v tabeli. Da določimo izračun povprečij po vrsticah moramo nastaviti vrednost argumenta `axis` na `1`. Spomnimo se namreč, da bi običajna uporaba metode `mean` računala povprečja po stolpcih in ne po vrsticah, kot rabimo v tem primeru.

Na koncu še s klicem funkcije [`merge`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) dodamo vsaki vrstici telefonsko številko iz tabele `studenti` in nato z ustreznim indeksiranjem izberemo spremenljivke (stolpce), ki jih hočemo ohraniti v končni različici tabele `so_objava`.

## Naloge

1. Napiši program v `pandas`, ki na osnovi podatkov iz tabel `studenti` in `studenti_ocene` izpiše ime študenta ali študentke z najboljšim povprečnim rezultatom na kolokvijih.

1. Sestavi tabelo, ki za posamezen kolokvij poroča razliko med najboljšim in najslabšim rezultatom študentov.

1. V tabeli z rezultati kolokvijev, ki smo jo sestavili med predavanji, dodaj naslednje tri spremenljivke.

    * `skupaj`: skupno število doseženih točk študenta (študentke) na kolokvijih;
  
    * `povprecje`: povprečno število doseženih točk študenta (študentke) na kolokvijih;
  
    * `ocena`: dosežena ocena študenta (študentke), pri čemer upoštevate običajno ocenjevalno lestvico, kjer je 10 ocena za povprečen rezultat nad 90 točk, 9 za povprečje nad 80, 8 za povprečje nad 70, 7 za povprečje nad 60, 6 za povprečje nad 50 in 5 za povprečje največ 50.
    
1. Sestavi tabelo funkcije $f(x,y)=\sin(x^2+y^2)$ za vse vrednosti $x$ in $y$ na intervalu $[-2,2]$ vzorčene s korakom $0.1$, torej $\{-2, -1.9, -1.8, \ldots, 2\}$. Iz te tabele za podano vrednost $\varepsilon$ sestavi seznam treh podatkovnih tabel: prva naj vsebuje tiste vrstice, za katere velja $z = f(x, y) < -\varepsilon$, druga vrstice $|z| \leq \varepsilon$ in tretja vrstice $z > \varepsilon$.
    
1. Spletna stran [Covid-19 sledilnik](https://covid-19.sledilnik.org/) ponuja dnevno osvežene podatke o stanju epidemije Covid-19 v Sloveniji. Podatki o dnevnem številu okužb ponujajo tudi skozi spletne storitve [GitHub](https://github.com/sledilnik/data/blob/master/csv/stats.csv): ti podatki so v obliki CSV dostopni na naslovu [https://raw.githubusercontent.com/sledilnik/data/master/csv/stats.csv](https://raw.githubusercontent.com/sledilnik/data/master/csv/stats.csv).

    Napiši program v R-ju, ki iz teh podatkov sestavi podatkovno tabelo z urejenimi dnevnimi podatki o številu okužb v celotni Sloveniji. Podatkovna tabela naj vsebuje naslednji dve spremenljivki:
    
    * `datum` je dimenzijska spremenjljvika z datumom opazovanega števila okužb;
    
    * `kumulativne_okuzbe` je merjena spremenljivka, ki za izbrano regijo poda kumulatinvo število okužb Covid-19 do izbranega datuma.
    
    Iz te osnovne tabele podatkov sestavi novo podatkovno tabelo, ki za posamezni mesec v opazovanem obdobju izračuna maksimalno število okužb v enem dnevu tega meseca. Nova tabela naj torej vsebuje dve spremenljivki:
    
    * `mesec` je dimenzijska spremenljivka, ki določi mesec opazovanja;
    
    * `max_dnevne_okuzbe` je numerična spremenljivka, ki poda maksimalno število dnevnih okužb v izbranem mesecu.
    
    Opomba in namig: Ta naloga je nadaljevanje naloge iz zvezka za predavanja o urejanju podatkov.